In [30]:
import pandas as pd

# READ DATASETS

In [31]:
folder = "./datasets/"
output_folder = "./datasets_models/"

### STATIONS DATASET


In [32]:
df_comarcas = pd.read_csv(folder + "t15902.csv", sep=";", header=0)

In [33]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [34]:

df_mesures = pd.read_csv(folder + "datos_temp.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [35]:
df_stations.comarca.unique()

array(['Alt Camp', 'Alt Empordà', 'Alt Penedès', 'Alt Urgell',
       'Alta Ribagorça', 'Anoia', 'Bages', 'Baix Camp', 'Baix Ebre',
       'Baix Empordà', 'Baix Llobregat', 'Baix Penedès', 'Barcelonès',
       'Berguedà', 'Cerdanya', 'Conca de Barberà', 'Garraf', 'Garrigues',
       'Garrotxa', 'Gironès', 'Maresme', 'Montsià', 'Noguera', 'Osona',
       'Pallars Jussà', 'Pallars Sobirà', "Pla d'Urgell",
       "Pla de l'Estany", 'Priorat', "Ribera d'Ebre", 'Ripollès',
       'Segarra', 'Segrià', 'Selva', 'Solsonès', 'Tarragonès',
       'Terra Alta', 'Urgell', "Val d'Aran", 'Vallès Occidental',
       'Vallès Oriental'], dtype=object)

In [36]:
df_comarcas.Comarca.unique()

array(['Alt Camp', 'Alt Empordà', 'Alt Penedès', 'Alt Urgell',
       'Alta Ribagorça', 'Anoia', 'Aran', 'Bages', 'Baix Camp',
       'Baix Ebre', 'Baix Empordà', 'Baix Llobregat', 'Baix Penedès',
       'Barcelonès', 'Berguedà', 'Cerdanya', 'Conca de Barberà', 'Garraf',
       'Garrigues', 'Garrotxa', 'Gironès', 'Maresme', 'Moianès',
       'Montsià', 'Noguera', 'Osona', 'Pallars Jussà', 'Pallars Sobirà',
       "Pla d'Urgell", "Pla de l'Estany", 'Priorat', "Ribera d'Ebre",
       'Ripollès', 'Segarra', 'Segrià', 'Selva', 'Solsonès', 'Tarragonès',
       'Terra Alta', 'Urgell', 'Vallès Occidental', 'Vallès Oriental',
       'Cataluña'], dtype=object)

# PROCESS DATA

In [37]:
codiE_por_comarca = {}
for comarca, group in df_stations.groupby('comarca'):
    codiE_por_comarca[comarca] = list(group['codiE'].unique())

In [38]:
comarcas_bcn = ['Vallès Oriental', 'Alt Penedès', 'Baix Llobregat', 'Barcelonès', 'Garraf', 'Maresme', 'Vallès Occidental']
codiE_comarca = [codiE_por_comarca[i] for i in comarcas_bcn]

In [39]:
data_comarca = []
for i in comarcas_bcn:
    df_filtrado = df_mesures[df_mesures['codi_estacio'].isin(codiE_por_comarca[i])]
    data_comarca.append(df_filtrado.reset_index(drop=True))

In [40]:
for comarca in data_comarca:
    comarca.data_lectura = comarca.data_lectura.str.split('T').str[0]

In [41]:
media_por_dia_comarca = []
for comarca in data_comarca:
    comarca['data_lectura'] = pd.to_datetime(comarca['data_lectura'])
    media_por_dia_comarca.append(comarca.groupby([comarca['data_lectura'].dt.date, 'codi_variable'])['valor_lectura'].mean().reset_index())

In [42]:
nuevo_orden = ["comarca", "codi_variable", "data_lectura", "valor_lectura"]
for i in range(len(media_por_dia_comarca)):
    media_por_dia_comarca[i]["comarca"] = comarcas_bcn[i]
    media_por_dia_comarca[i] = media_por_dia_comarca[i][nuevo_orden].reset_index(drop=True)

In [43]:
df_concatenado = pd.concat(media_por_dia_comarca).reset_index(drop=True)
df_concatenado.to_csv(output_folder + "mean_temp_x_day.csv", index=False)

In [44]:
df_concatenado

,comarca,codi_variable,data_lectura,valor_lectura
0,Vallès Oriental,32,2009-01-01,8.059722
1,Vallès Oriental,32,2009-01-02,7.416667
2,Vallès Oriental,32,2009-01-03,5.665278
3,Vallès Oriental,32,2009-01-04,4.345833
4,Vallès Oriental,32,2009-01-05,3.138824
...,...,...,...,...
37095,Vallès Occidental,32,2023-07-05,21.832143
37096,Vallès Occidental,32,2023-07-06,22.760119
37097,Vallès Occidental,32,2023-07-07,23.723214
37098,Vallès Occidental,32,2023-07-08,25.869048
